# StockVision Data Fetcher
Yahoo Finance APIから株価データを取得してCSV形式で出力するノートブック

## 特徴
- レート制限対策付き
- 大量データの安全な取得
- エラーハンドリング
- Google Driveとの連携

In [ ]:
# 必要なライブラリのインストール
!pip install yfinance pandas numpy

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

print(f"データ取得開始時刻: {datetime.now()}")

In [ ]:
# 取得対象銘柄の設定
# 日本株（東証）
japanese_stocks = [
    "7203.T",  # トヨタ自動車
    "9984.T",  # ソフトバンクG
    "6758.T",  # ソニー
    "4063.T",  # 信越化学
    "8306.T",  # 三菱UFJ
    "9983.T",  # ファーストリテ
    "6861.T",  # キーエンス
    "4519.T",  # 中外製薬
    "8035.T",  # 東京エレクトロン
    "7974.T"   # 任天堂
]

# 米国株（参考用）
us_stocks = [
    "AAPL",    # Apple
    "GOOGL",   # Alphabet
    "MSFT",    # Microsoft
    "AMZN",    # Amazon
    "TSLA"     # Tesla
]

# メイン取得対象
target_symbols = japanese_stocks
print(f"取得対象: {len(target_symbols)}銘柄")
print(target_symbols)

In [ ]:
def fetch_stock_data_safe(symbols, period="2y", delay=2):
    """
    レート制限を考慮した安全な株価データ取得
    
    Args:
        symbols: 銘柄リスト
        period: 取得期間 (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)
        delay: 銘柄間の遅延秒数
    """
    results = []
    failed_symbols = []
    
    for i, symbol in enumerate(symbols):
        try:
            print(f"取得中 ({i+1}/{len(symbols)}): {symbol}")
            
            # Tickerオブジェクト作成
            ticker = yf.Ticker(symbol)
            
            # 履歴データ取得
            hist = ticker.history(period=period)
            
            # 銘柄情報取得
            try:
                info = ticker.info
                company_name = info.get('longName', info.get('shortName', 'Unknown'))
                market_cap = info.get('marketCap')
                pe_ratio = info.get('trailingPE')
            except:
                company_name = 'Unknown'
                market_cap = None
                pe_ratio = None
            
            # データが存在する場合のみ処理
            if not hist.empty:
                # DataFrame処理
                hist_reset = hist.reset_index()
                hist_reset['symbol'] = symbol
                hist_reset['company_name'] = company_name
                hist_reset['market_cap'] = market_cap
                hist_reset['pe_ratio'] = pe_ratio
                hist_reset['updated_at'] = datetime.now().isoformat()
                
                # カラム名正規化
                hist_reset.columns = [col.lower() if isinstance(col, str) else col for col in hist_reset.columns]
                
                results.append(hist_reset)
                print(f"✓ {symbol}: {len(hist_reset)}件のデータを取得")
            else:
                print(f"⚠ {symbol}: データが見つかりません")
                failed_symbols.append(symbol)
            
            # レート制限対策の遅延
            if i < len(symbols) - 1:  # 最後の銘柄以外
                time.sleep(delay)
                
        except Exception as e:
            print(f"❌ {symbol}: エラー - {e}")
            failed_symbols.append(symbol)
            time.sleep(delay * 2)  # エラー時は長めに待機
    
    if results:
        combined_df = pd.concat(results, ignore_index=True)
        print(f"\n✅ 取得完了: {len(results)}銘柄, {len(combined_df)}レコード")
    else:
        combined_df = pd.DataFrame()
        print("❌ データ取得に失敗しました")
    
    if failed_symbols:
        print(f"⚠ 失敗した銘柄: {failed_symbols}")
    
    return combined_df, failed_symbols

In [ ]:
# データ取得実行
print("=== データ取得開始 ===")
start_time = datetime.now()

stock_data_df, failed = fetch_stock_data_safe(
    target_symbols, 
    period="2y",  # 2年分
    delay=3       # 3秒間隔
)

end_time = datetime.now()
duration = end_time - start_time

print(f"\n=== 取得結果 ===")
print(f"実行時間: {duration}")
print(f"総レコード数: {len(stock_data_df)}")
print(f"銘柄数: {stock_data_df['symbol'].nunique() if not stock_data_df.empty else 0}")
print(f"失敗数: {len(failed)}")

In [ ]:
# データ確認
if not stock_data_df.empty:
    print("=== データサマリー ===")
    print(stock_data_df.info())
    print("\n=== 先頭5行 ===")
    print(stock_data_df.head())
    print("\n=== 銘柄別レコード数 ===")
    print(stock_data_df['symbol'].value_counts())
else:
    print("データが空です")

In [ ]:
# CSVファイル出力
if not stock_data_df.empty:
    filename = f"stock_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    stock_data_df.to_csv(filename, index=False, encoding='utf-8')
    print(f"✅ CSVファイルを出力しました: {filename}")
    
    # ファイルサイズ確認
    import os
    file_size = os.path.getsize(filename) / 1024 / 1024  # MB
    print(f"ファイルサイズ: {file_size:.2f} MB")
    
    # Google Driveにマウント（オプション）
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Google Driveにコピー
        import shutil
        drive_path = f'/content/drive/My Drive/{filename}'
        shutil.copy(filename, drive_path)
        print(f"✅ Google Driveにもコピーしました: {drive_path}")
    except Exception as e:
        print(f"Google Drive連携エラー: {e}")
        
else:
    print("❌ 出力するデータがありません")

In [ ]:
# 最新価格データの生成（現在価格用）
if not stock_data_df.empty:
    print("=== 最新価格データ生成 ===")
    
    latest_prices = []
    
    for symbol in stock_data_df['symbol'].unique():
        symbol_data = stock_data_df[stock_data_df['symbol'] == symbol].sort_values('date')
        if len(symbol_data) >= 2:
            latest = symbol_data.iloc[-1]
            previous = symbol_data.iloc[-2]
            
            price_change = latest['close'] - previous['close']
            price_change_pct = (price_change / previous['close']) * 100
            
            latest_prices.append({
                'symbol': symbol,
                'company_name': latest['company_name'],
                'current_price': latest['close'],
                'previous_close': previous['close'],
                'price_change': price_change,
                'price_change_pct': price_change_pct,
                'volume': latest['volume'],
                'market_cap': latest['market_cap'],
                'pe_ratio': latest['pe_ratio'],
                'last_updated': latest['updated_at']
            })
    
    latest_df = pd.DataFrame(latest_prices)
    latest_filename = f"latest_prices_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    latest_df.to_csv(latest_filename, index=False, encoding='utf-8')
    
    print(f"✅ 最新価格CSVを出力しました: {latest_filename}")
    print(latest_df.head())

## 使用方法

1. このノートブックを実行してCSVファイルを生成
2. 生成されたCSVをダウンロード or Google Driveから取得
3. StockVisionアプリの `/api/csv/upload` エンドポイントでアップロード
4. または `/api/csv/download-from-colab` でColab URLから直接取得

## 自動化案
- Google Colabのスケジュール実行
- GitHub Actionsとの連携
- Google Drive API経由の自動取得